#### Лабораторная работа № 1. Разработка нейросетевых функций. Операция Convolution 2D
#### БВТ2003 Никитин Степан Романович

In [116]:
import torch
import numpy as np
from typing import Union

In [117]:
class Conv2D:
    def __init__(self, in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple] = 1) -> None:
        """
        Конструктор класса Conv2D
        """
        self.in_channels = in_channels     # количество входных каналов (каналов входного тензора)
        self.out_channels = out_channels   # количество выходных каналов (каналов результата свертки)
        self.kernel_size = kernel_size     # размер ядра свертки (предполагается квадратное ядро)
        self.stride = stride               # шаг свертки

        # преобразование kernel_size в кортеж, если он целое число
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        # инициализация весов (ядер свертки)
        self.weights = np.random.randn(out_channels, in_channels, *kernel_size)
        self.bias = np.zeros((out_channels,))

    def forward(self, input_tensor: np.ndarray) -> np.ndarray:
        """
        Метод для выполнения операции свертки
        """
        in_channels, input_height, input_width = input_tensor.shape
        kernel_size = self.kernel_size

        # преобразование kernel_size в кортеж, если он целое число
        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        # проверка на корректные размеры входного тензора
        if input_height < kernel_size[0] or input_width < kernel_size[1]:
            raise ValueError("Неверный размер ядра для входного тензора")

        # вычисление размеров результата свертки
        if isinstance(self.stride, int):
            stride_h = stride_w = self.stride
        elif isinstance(self.stride, tuple) and len(self.stride) == 2:
            stride_h, stride_w = self.stride
        else:
            raise ValueError("Недопустимый формат шага. Должно быть целое число или кортеж из двух целых чисел")

        output_height = (input_height - kernel_size[0]) // stride_h + 1
        output_width = (input_width - kernel_size[1]) // stride_w + 1

        # инициализация результата свертки
        result = np.zeros((self.out_channels, output_height, output_width))

        # выполнение свертки
        for i in range(0, input_height - kernel_size[0] + 1, stride_h):
            for j in range(0, input_width - kernel_size[1] + 1, stride_w):
                # окно свертки
                window = input_tensor[:, i:i + kernel_size[0], j:j + kernel_size[1]]

                # вычисление свертки и добавление bias
                result[:, i // stride_h, j // stride_w] = np.sum(window * self.weights, axis=(1, 2, 3)) + self.bias

        return result

In [118]:
def run_test(in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple], 
             padding: Union[int, tuple], dilation: int, groups: int, bias: bool, padding_mode: str) -> None:
    """
    Функция для выполнения теста операции свертки
    """
    # создание экземпляра Conv2D
    conv_layer = Conv2D(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)

    # создание тензора для теста
    input_tensor = np.random.randn(in_channels, 10, 10)

    # выполнение свертки
    result = conv_layer.forward(input_tensor)

    print(f"Тест с параметрами:")
    print(f"  - in_channels: {in_channels}")
    print(f"  - out_channels: {out_channels}")
    print(f"  - kernel_size: {kernel_size}")
    print(f"  - stride: {stride}")
    print(f"  - padding: {padding}")
    print(f"  - dilation: {dilation}")
    print(f"  - groups: {groups}")
    print(f"  - bias: {bias}")
    print(f"  - padding_mode: {padding_mode}")
    print(f"\nРезультат свертки:\n\n {result}")
    print(f"\n{'-'*75}\n")

In [119]:
def test_params() -> None:
    """
    Функция для тестирования различных параметров операции свертки
    """
    # параметры тестирования для первой группы
    run_test(in_channels=8, out_channels=20, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')

    # параметры тестирования для второй группы
    run_test(in_channels=4, out_channels=16, kernel_size=2, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='replicate')

    # параметры тестирования для третьей группы
    run_test(in_channels=8, out_channels=4, kernel_size=2, stride=(2, 3), padding=0, dilation=1, groups=2, bias=True, padding_mode='replicate')

    # параметры тестирования для четвертой группы
    run_test(in_channels=5, out_channels=32, kernel_size=(3, 2), stride=(4, 3), padding=1, dilation=2, groups=1, bias=True, padding_mode='zeros')

In [120]:
# параметры тестирования для дополнительной группы
input_tensor = np.random.randn(3, 5, 5)
conv_layer = Conv2D(in_channels=3, out_channels=2, kernel_size=3, stride=(2, 2))
output_tensor = conv_layer.forward(input_tensor)

print(f"Результат свертки:\n {output_tensor}")

Результат свертки:
 [[[-3.31509986e+00 -1.05183147e+01]
  [-2.94842389e+00  8.59191873e-02]]

 [[-6.77251703e+00  3.14028827e+00]
  [-1.20968534e+00 -5.30691268e-03]]]


In [121]:
test_params()

Тест с параметрами:
  - in_channels: 8
  - out_channels: 20
  - kernel_size: 3
  - stride: 1
  - padding: 0
  - dilation: 1
  - groups: 2
  - bias: True
  - padding_mode: zeros

Результат свертки:

 [[[-12.20362284   4.61641224  -3.17443033 ... -24.55494153   3.12795656
     4.81712816]
  [-12.13519978  -8.96658545  -7.46859494 ...  -0.32071867 -12.58674838
    -0.30160208]
  [ -8.60980511 -24.71581217 -11.8156218  ... -12.35536706   1.32140973
   -16.53894113]
  ...
  [  1.45196737   2.66005198 -14.35616703 ...  -6.45722651 -13.57796649
     6.3201218 ]
  [  7.20078555  -1.54103493 -15.38246985 ...  -1.95579554  -9.42638606
     4.97567345]
  [-13.85172565   8.88989395   4.95656419 ...  -6.01853158   8.23018859
     5.60689784]]

 [[  8.85865901  -9.58685936 -13.51064666 ... -26.70703942 -15.7411194
    -5.1117525 ]
  [ -8.6035887   -1.80866139 -13.89622206 ...  -7.12267887   6.48369341
    -3.28807994]
  [ -4.45514928 -21.16076841  -0.44695783 ... -18.91109119   7.22850474
    -5.526